##Initialize Project

In [1]:
!pip install -U 'scikit-learn<0.24'
!pip install sklearn-crfsuite
!pip install tqdm
# YOU NEED TO RESTART THE RUNTIME!!!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 8.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 7.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Run this cell to mount your drive to this notebook in order to read the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import json
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## Read Dataset

In [4]:
# Put the folder path where the datasets are located
PATH = "drive/My Drive/Colab Notebooks/CS445_NLP/Project #2/Datasets/"

In [5]:
def read_data(data):
  temp = []
  with open(data) as f:
        text = f.read()
  paragraph_list = text.split('\n\n')
  for sent in paragraph_list:
    if sent != "-DOCSTART- -X- -X- O":
      if sent != "":
        items = sent.split('\n')
        result = tuple(tuple(i.split(' ')) for i in items)
        temp.append(result)
        #print(tuple(tuple(i.split(' ')) for i in items))
  return temp

In [6]:
final = read_data(PATH+"train.txt")
final[:3]
#DOSSTARTLARI at

[(('EU', 'NNP', 'B-NP', 'B-ORG'),
  ('rejects', 'VBZ', 'B-VP', 'O'),
  ('German', 'JJ', 'B-NP', 'B-MISC'),
  ('call', 'NN', 'I-NP', 'O'),
  ('to', 'TO', 'B-VP', 'O'),
  ('boycott', 'VB', 'I-VP', 'O'),
  ('British', 'JJ', 'B-NP', 'B-MISC'),
  ('lamb', 'NN', 'I-NP', 'O'),
  ('.', '.', 'O', 'O')),
 (('Peter', 'NNP', 'B-NP', 'B-PER'), ('Blackburn', 'NNP', 'I-NP', 'I-PER')),
 (('BRUSSELS', 'NNP', 'B-NP', 'B-LOC'), ('1996-08-22', 'CD', 'I-NP', 'O'))]

In [7]:
# read data with your custom function
train_data = read_data(PATH+"train.txt")
val_data = read_data(PATH+"valid.txt")
test_data = read_data(PATH+"test.txt")

In [8]:
print("Length of the train data: ", len(train_data))
print("Length of the validation data: ", len(val_data))
print("Length of the test data: ", len(test_data))

Length of the train data:  14041
Length of the validation data:  3250
Length of the test data:  3453


In [9]:
# Train data
print(train_data[:3])

[(('EU', 'NNP', 'B-NP', 'B-ORG'), ('rejects', 'VBZ', 'B-VP', 'O'), ('German', 'JJ', 'B-NP', 'B-MISC'), ('call', 'NN', 'I-NP', 'O'), ('to', 'TO', 'B-VP', 'O'), ('boycott', 'VB', 'I-VP', 'O'), ('British', 'JJ', 'B-NP', 'B-MISC'), ('lamb', 'NN', 'I-NP', 'O'), ('.', '.', 'O', 'O')), (('Peter', 'NNP', 'B-NP', 'B-PER'), ('Blackburn', 'NNP', 'I-NP', 'I-PER')), (('BRUSSELS', 'NNP', 'B-NP', 'B-LOC'), ('1996-08-22', 'CD', 'I-NP', 'O'))]


In [10]:
# Validation data
print(val_data[:5])

[(('CRICKET', 'NNP', 'B-NP', 'O'), ('-', ':', 'O', 'O'), ('LEICESTERSHIRE', 'NNP', 'B-NP', 'B-ORG'), ('TAKE', 'NNP', 'I-NP', 'O'), ('OVER', 'IN', 'B-PP', 'O'), ('AT', 'NNP', 'B-NP', 'O'), ('TOP', 'NNP', 'I-NP', 'O'), ('AFTER', 'NNP', 'I-NP', 'O'), ('INNINGS', 'NNP', 'I-NP', 'O'), ('VICTORY', 'NN', 'I-NP', 'O'), ('.', '.', 'O', 'O')), (('LONDON', 'NNP', 'B-NP', 'B-LOC'), ('1996-08-30', 'CD', 'I-NP', 'O')), (('West', 'NNP', 'B-NP', 'B-MISC'), ('Indian', 'NNP', 'I-NP', 'I-MISC'), ('all-rounder', 'NN', 'I-NP', 'O'), ('Phil', 'NNP', 'I-NP', 'B-PER'), ('Simmons', 'NNP', 'I-NP', 'I-PER'), ('took', 'VBD', 'B-VP', 'O'), ('four', 'CD', 'B-NP', 'O'), ('for', 'IN', 'B-PP', 'O'), ('38', 'CD', 'B-NP', 'O'), ('on', 'IN', 'B-PP', 'O'), ('Friday', 'NNP', 'B-NP', 'O'), ('as', 'IN', 'B-PP', 'O'), ('Leicestershire', 'NNP', 'B-NP', 'B-ORG'), ('beat', 'VBD', 'B-VP', 'O'), ('Somerset', 'NNP', 'B-NP', 'B-ORG'), ('by', 'IN', 'B-PP', 'O'), ('an', 'DT', 'B-NP', 'O'), ('innings', 'NN', 'I-NP', 'O'), ('and', 'CC',

In [11]:
# Test Data
print(test_data[:5])

[(('SOCCER', 'NN', 'B-NP', 'O'), ('-', ':', 'O', 'O'), ('JAPAN', 'NNP', 'B-NP', 'B-LOC'), ('GET', 'VB', 'B-VP', 'O'), ('LUCKY', 'NNP', 'B-NP', 'O'), ('WIN', 'NNP', 'I-NP', 'O'), (',', ',', 'O', 'O'), ('CHINA', 'NNP', 'B-NP', 'B-PER'), ('IN', 'IN', 'B-PP', 'O'), ('SURPRISE', 'DT', 'B-NP', 'O'), ('DEFEAT', 'NN', 'I-NP', 'O'), ('.', '.', 'O', 'O')), (('Nadim', 'NNP', 'B-NP', 'B-PER'), ('Ladki', 'NNP', 'I-NP', 'I-PER')), (('AL-AIN', 'NNP', 'B-NP', 'B-LOC'), (',', ',', 'O', 'O'), ('United', 'NNP', 'B-NP', 'B-LOC'), ('Arab', 'NNP', 'I-NP', 'I-LOC'), ('Emirates', 'NNPS', 'I-NP', 'I-LOC'), ('1996-12-06', 'CD', 'I-NP', 'O')), (('Japan', 'NNP', 'B-NP', 'B-LOC'), ('began', 'VBD', 'B-VP', 'O'), ('the', 'DT', 'B-NP', 'O'), ('defence', 'NN', 'I-NP', 'O'), ('of', 'IN', 'B-PP', 'O'), ('their', 'PRP$', 'B-NP', 'O'), ('Asian', 'JJ', 'I-NP', 'B-MISC'), ('Cup', 'NNP', 'I-NP', 'I-MISC'), ('title', 'NN', 'I-NP', 'O'), ('with', 'IN', 'B-PP', 'O'), ('a', 'DT', 'B-NP', 'O'), ('lucky', 'JJ', 'I-NP', 'O'), ('2-1

# Create Gazetteer

In [14]:
def href_regex_extractor(text):
  regex = re.compile("href=\\\"(?:%\d+)?((?:([A-ZIÜŞÖÇ][A-Za-zığüşöç]*))(?:%\d+)?)*\\\"&gt;(\s?[A-ZIÜŞÖÇ][A-Za-zığüşöç]*\s?)*&lt;")
  temp = []
  for m in re.finditer(regex, text): 
    temp.append(m.group())
  return temp

In [15]:
# Load wikipedia pages and extract the href expressions to a list.
from tqdm import tqdm

dir = "/content/drive/My Drive/Colab Notebooks/CS445_NLP/Project #2/Wikipedia_Pages/wikipedia_pages/wikipedia_pages"

hrefs = []
count = 0
for file in tqdm(os.listdir(dir)):
  with open(dir+ '/' + file, 'r') as file:
    if count == 999:
      print("\n\nThis file cannot be processed:")
      print(file)
      count+=1
    else:
      data = json.load(file)
      result = href_regex_extractor(data.get('text'))
      result = ",".join(result)
      hrefs.append(result)
      count+=1
        

  6%|▌         | 1129/20000 [00:33<00:29, 639.85it/s]



This file cannot be processed:
<_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/CS445_NLP/Project #2/Wikipedia_Pages/wikipedia_pages/wikipedia_pages/37138.json' mode='r' encoding='UTF-8'>


100%|██████████| 20000/20000 [01:06<00:00, 299.80it/s]


In [16]:
print(len(hrefs))
hrefs[:10]




19999


['href="Stanley%20Kramer"&gt;Stanley Kramer&lt;,href="Marlon%20Brando"&gt;Marlon Brando&lt;,href="American%20Motorcyclist%20Association"&gt;American Motorcyclist Association&lt;,href="VHS"&gt;VHS&lt;,href="Betamax"&gt;Betamax&lt;,href="DVD"&gt;DVD&lt;,href="Sony%20Pictures"&gt;Sony Pictures&lt;,href="Mill%20Creek%20Entertainment"&gt;Mill Creek Entertainment&lt;,href="Rotten%20Tomatoes"&gt;Rotten Tomatoes&lt;,href="Dave%20Kehr"&gt;Dave Kehr&lt;,href="Chicago%20Reader"&gt;Chicago Reader&lt;,href="Paddington"&gt;Paddington&lt;,href="Triumph%20Engineering"&gt;Triumph&lt;,href="Gil%20Stratton"&gt;Gil Stratton Jr&lt;,href="American%20Film%20Institute"&gt;American Film Institute&lt;,href="James%20Dean"&gt;James Dean&lt;,href="Elvis%20Presley"&gt;Elvis Presley&lt;,href="The%20Beatles%20Anthology"&gt;The Beatles Anthology&lt;,href="Black%20Rebel%20Motorcycle%20Club"&gt;Black Rebel Motorcycle Club&lt;,href="Everybody%20Loves%20Raymond"&gt;Everybody Loves Raymond&lt;,href="The%20Simpsons"&gt;The 

In [17]:
# REGEX Extract Title
def regex_extract_title(text):
  regex = re.compile("\"((?:%\d+)?[A-ZIÜŞÖÇ0-9][A-Za-zığüşöç0-9]*)*")
  temp = []
  for m in re.finditer(regex, text):

    regex_1 = re.compile("\"")
    subst_1 = ""
    result_1 = re.sub(regex_1, subst_1, m.group(0), 0)

    regex_2 = re.compile("(%\d+)")
    subst_2 = " "
    result_2 = re.sub(regex_2, subst_2, result_1, 0)
    temp.append(result_2)

  return temp
  


count_row = 0
extracted_titles = []

for row in tqdm(hrefs):
  #print(i)
  result_regex = regex_extract_title(row)
  for i in result_regex:
    if i  != '':
      extracted_titles.append(i)

100%|██████████| 19999/19999 [00:03<00:00, 6221.50it/s]


In [18]:
#REGEX Extract Anchor
def regex_extract_anchor(text):
  regex = re.compile("gt;([A-ZIÜŞÖÇ0-9][A-Za-zığüşöç0-9]*( )?)*")
  temp = []
  for m in re.finditer(regex, text):
    regex_2 = re.compile("gt;")
    subst_2 = ""
    result = re.sub(regex_2, subst_2, m.group(0), 0)
    temp.append(result)
  return temp


count_row_anchor = 0
extracted_anchors = []

for row in tqdm(hrefs):
  result_regex_anchor = regex_extract_anchor(row)
  for i in result_regex_anchor:
    extracted_anchors.append(i)

100%|██████████| 19999/19999 [00:01<00:00, 19911.71it/s]


In [19]:
# Extraction Results from JSON Files
# Titles
print("Amount of the extracted titles: ", len(extracted_titles))
print("Sample Extracted Titles:\n")
print(extracted_titles[10:18])

#-------------------------------------------------------------------------------
print("\n\n")
# Anchors
print("Amount of the extracted anchors: ", len(extracted_anchors))
print("Sample ExtractedAnchors:\n")
print(extracted_anchors[10:18])

Amount of the extracted titles:  331842
Sample Extracted Titles:

['Chicago Reader', 'Paddington', 'Triumph Engineering', 'Gil Stratton', 'American Film Institute', 'James Dean', 'Elvis Presley', 'The Beatles Anthology']



Amount of the extracted anchors:  331842
Sample ExtractedAnchors:

['Chicago Reader', 'Paddington', 'Triumph', 'Gil Stratton Jr', 'American Film Institute', 'James Dean', 'Elvis Presley', 'The Beatles Anthology']


In [20]:
# Create Gazetteer Word List with all unique words
MyGazetteer = []

# Append titles and anchors to the Gazetteer
MyGazetteer = extracted_titles + extracted_anchors
print("Size of the Gazetteer after adding titles and anchors: ", len(MyGazetteer))

# Remove the dublicates from MyGazetteer
UniqueGazetteer = {}

for i in MyGazetteer:
  UniqueGazetteer[i] = ""

print("Unique Words in the Gazetteer: ", len(UniqueGazetteer.keys()))
print("Gazetteer:")
MyGazetteer[:20]

Size of the Gazetteer after adding titles and anchors:  663684
Unique Words in the Gazetteer:  165610
Gazetteer:


['Stanley Kramer',
 'Marlon Brando',
 'American Motorcyclist Association',
 'VHS',
 'Betamax',
 'DVD',
 'Sony Pictures',
 'Mill Creek Entertainment',
 'Rotten Tomatoes',
 'Dave Kehr',
 'Chicago Reader',
 'Paddington',
 'Triumph Engineering',
 'Gil Stratton',
 'American Film Institute',
 'James Dean',
 'Elvis Presley',
 'The Beatles Anthology',
 'Black Rebel Motorcycle Club',
 'Everybody Loves Raymond']

# Models

## Conditional Random Fields (CRF)

### Extract features for CRF

In [24]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

import sklearn_crfsuite
from sklearn.metrics import make_scorer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
def wordShape(text):
    t1 = re.sub('[A-Z]', 'X',text)
    t2 = re.sub('[a-z]', 'x', t1)
    return re.sub('[0-9]', 'd', t2)

In [26]:
def shortWordShape(text):
    t1 = re.sub('([A-Z]+)', 'X',text)
    t2 = re.sub('([a-z]+)', 'x', t1)
    return re.sub('([0-9]+)', 'd', t2)

In [27]:
def has_number(text):
    return bool(re.search(r'\d', text))

In [28]:
def has_hyphen(text):
    return bool(re.search(r'-', text))

In [29]:
def is_upper_has_digit_has_dash(text):
  result = False
  if text.isupper():
    if has_number(text):
      if bool(re.search(r'-', text)):
        result = True

  return result

In [30]:
def is_stopword(text):
  result = False
  stop_words = set(stopwords.words("english"))
  for word in stop_words:
    if text == word:
      result = True
    else:
      result = False
      
  return result

In [31]:
def is_in_gazetteer(text):
  for word in MyGazetteer:
    result = False
    if text == word:
      return True
    else:
      result = False
      
  return result

In [32]:
from nltk.corpus.reader.wordnet import WordNetError
# create a function to extract features for each token

def token2features(sentence: list, idx: int) -> dict:
  ps = PorterStemmer()
  word = sentence[idx][0]
  features = {
      'stem': "None",
      'POS_tag': "None",
      'Chunk_tag': "None",
      'is_BOS': "None",
      'is_EOS': "None",
      'is_first_letter_uppercase': "None",
      'word_shape': "None",
      'short_word_shape': "None",
      'has_number': "None",
      'has_hyphen': "None",
      'is_upper_has_digit_has_dash': "None",
      'prefix':"None",
      'suffix': "None",
      'is_all_upper': "None",
      'is_stopword': "None",

      # Neighbor Words
      # Word Before
      'word_previous': "None",
      'word_previous_shape': "None",
      'word_previous_short_shape': "None",

      # Word After
      'word_after': "None",
      'word_after_shape': "None",
      'word_after_short_shape': "None",

      'is_in_gazetteer':"None"
  }

  features.update({'stem' : ps.stem(word)}),
  features.update({'POS_tag' : sentence[idx][1]})
  features.update({'Chunk_tag' : sentence[idx][2]})

  if(len(sentence) <= 2):
    features.update({
        'is_BOS' : True,
        'is_EOS' : True
    })
  

  else:
    # FIRST WORD
    if idx == 0:
      features.update({
          'is_BOS' : True,
          'is_EOS' : False,
          'word_after' : sentence[idx+1][0],
          'word_after_shape' : wordShape(sentence[idx+1][0]),
          'word_after_short_shape' : shortWordShape(sentence[idx+1][0]) 
      })
        

    # LAST WORD
    elif idx == len(sentence)-1:
      features.update({
          'is_BOS' : False,
          'is_EOS' : True,
          'word_previous' : sentence[idx-1][0],
          'word_previous_shape' : wordShape(sentence[idx-1][0]),
          'word_previous_short_shape' : shortWordShape(sentence[idx-1][0])
      })


    else:
      #  WORD IN THE MIDDLE
      features.update({
          'is_BOS' : False,
          'is_EOS' : False,
          'word_previous' : sentence[idx-1][0],
          'word_after' : sentence[idx+1][0],

          'word_previous_shape' : wordShape(sentence[idx-1][0]),
          'word_previous_short_shape' : shortWordShape(sentence[idx-1][0]),

          'word_after_shape' : wordShape(sentence[idx+1][0]),
          'word_after_short_shape' : shortWordShape(sentence[idx+1][0])
          
      })
      
  features.update({
      'is_first_letter_uppercase' : word[0].isupper(),
      'word_shape' : wordShape(word),
      'short_word_shape' : shortWordShape(word),
      'has_number' : has_number(word),
      'has_hyphen' : has_hyphen(word),
      'is_upper_has_digit_has_dash' : is_upper_has_digit_has_dash(word),
      'prefix' : word[:4],
      'suffix' : word[-4:],
      'is_all_upper' : word.isupper(),
      'is_stopword' : is_stopword(word),
      'is_in_gazetteer' : is_in_gazetteer(word)
  })
  return features


In [33]:
# define function to process each token given a sentence
def sent2features(sentence: list) -> list:
  return [token2features(sentence, i) for i in range(len(sentence))]

# get named entity labels from the sentence
def sent2labels(sentence: list) -> list:
  NER_labels = []
  for word in sentence:
    NER_labels.append(word[3])
  return NER_labels

In [34]:
# prepare inputs and labels

train_sents = [sent2features(s) for s in tqdm(train_data)]
val_sents = [sent2features(s) for s in tqdm(val_data)]
test_sents = [sent2features(s) for s in tqdm(test_data)]


100%|██████████| 3453/3453 [27:43<00:00,  2.08it/s]


In [35]:
train_labels = [sent2labels(s) for s in tqdm(train_data)]
val_labels = [sent2labels(s) for s in tqdm(val_data)]
test_labels = [sent2labels(s) for s in tqdm(test_data)]

100%|██████████| 3453/3453 [00:00<00:00, 241048.74it/s]


In [36]:
train_sents[1] # Peter Blackburn

[{'Chunk_tag': 'B-NP',
  'POS_tag': 'NNP',
  'has_hyphen': False,
  'has_number': False,
  'is_BOS': True,
  'is_EOS': True,
  'is_all_upper': False,
  'is_first_letter_uppercase': True,
  'is_in_gazetteer': True,
  'is_stopword': False,
  'is_upper_has_digit_has_dash': False,
  'prefix': 'Pete',
  'short_word_shape': 'Xx',
  'stem': 'peter',
  'suffix': 'eter',
  'word_after': 'None',
  'word_after_shape': 'None',
  'word_after_short_shape': 'None',
  'word_previous': 'None',
  'word_previous_shape': 'None',
  'word_previous_short_shape': 'None',
  'word_shape': 'Xxxxx'},
 {'Chunk_tag': 'I-NP',
  'POS_tag': 'NNP',
  'has_hyphen': False,
  'has_number': False,
  'is_BOS': True,
  'is_EOS': True,
  'is_all_upper': False,
  'is_first_letter_uppercase': True,
  'is_in_gazetteer': True,
  'is_stopword': False,
  'is_upper_has_digit_has_dash': False,
  'prefix': 'Blac',
  'short_word_shape': 'Xx',
  'stem': 'blackburn',
  'suffix': 'burn',
  'word_after': 'None',
  'word_after_shape': 'None

In [37]:
train_labels[1] # Peter Blackburn

['B-PER', 'I-PER']

###Initialize the CRF Model and GridSearchCV

In [33]:
# Initialize the CRF model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [34]:
# Set the hyperparameter space that will be scanned.
# TAKES SO MUCH TIME, THEREFORE I AM COMMENTING THIS PART
#crf= sklearn_crfsuite.CRF()

#grid_params_CRF = {
#  "c1": [0, 0.05, 0.25],
#  "c2": [0, 0.05, 0.25]
#}

# initialize GridSearchCV for CRF
#grid_search_CRF = GridSearchCV(
#                 estimator = crf,
#                 param_grid = grid_params_CRF,
#                 verbose = 2)


# fitting the model for grid search 
#grid_search_CRF.fit(train_sents, train_labels)


#print best parameter after tuning 
#print("Best Parameters: ", grid_search_CRF.best_params_)
  
# print how our model looks after hyper-parameter tuning 
#print("Model after hyper-parameter tuning: ", grid_search_CRF.cv_results_['params'][grid_search_CRF.best_index_])
#print("Best Estimator: ", grid_search_CRF.best_estimator_)

In [35]:
# initialize and train a crf model with best hyper-parameters
crf_best_estimator = sklearn_crfsuite.CRF(c1=0.05, c2=0.25, keep_tempfiles=None)

# fit your model
crf_best_estimator.fit(train_sents, train_labels)

# make predictions
preds_crf = crf_best_estimator.predict(val_sents)

# calculate f1-score and classification report for test using sklearn_crfsuite.metrics class
# evaluate on validation set
precision_crf = sklearn_crfsuite.metrics.flat_precision_score(val_labels, preds_crf, average = 'weighted')
recall_crf = sklearn_crfsuite.metrics.flat_recall_score(val_labels, preds_crf, average = 'weighted')
f1_crf = sklearn_crfsuite.metrics.flat_f1_score(val_labels, preds_crf, average = 'weighted')

print("Precision Score:", precision_crf, "\n",
      "Recall Score:", recall_crf, "\n",
      "F1 Score:", f1_crf)

Precision Score: 0.9788677529727192 
 Recall Score: 0.9791480082551303 
 F1 Score: 0.9789197238702536


In [36]:
def drop_dict_keys(sent_list, remove_features_list):
  for k in remove_features_list:
    for sentence in sent_list:
      for word in sentence:
        word.pop(k, None)
  return sent_list

In [37]:
def get_crf_results(train_sents_p, val_sents_p, train_labels_, val_labels_):
  crf_best_estimator = sklearn_crfsuite.CRF(c1=0.05, c2=0.25, keep_tempfiles=None)
  crf_best_estimator.fit(train_sents_p, train_labels_)
  crf_best_estimator_preds = crf_best_estimator.predict(val_sents_p)

  precisions_crf = sklearn_crfsuite.metrics.flat_precision_score(val_labels_, crf_best_estimator_preds, average = 'weighted')
  recalls_crf = sklearn_crfsuite.metrics.flat_recall_score(val_labels_, crf_best_estimator_preds, average = 'weighted')
  f1_scores_crf = sklearn_crfsuite.metrics.flat_f1_score(val_labels_, crf_best_estimator_preds, average = 'weighted')

  #results_add_feature_1by1_df = pd.DataFrame({"Features": features_for_add_1by1, "Precision": precisions_1by1, "Recall": recalls_1by1, "F1 Score": f1_scores_1by1})
  result = [precisions_crf, recalls_crf, f1_scores_crf]
  print(result)
  return result

###Give Features One By One to the CRF Model

In [38]:
#'stem'
import copy
features_remove_1 = ( 
            'POS_tag', 'Chunk_tag', 'is_BOS', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_1 = copy.deepcopy(train_sents)
val_sents_1 = copy.deepcopy(val_sents)
train_sents_1 = drop_dict_keys(train_sents_1, features_remove_1)
val_sents_1 = drop_dict_keys(val_sents_1, features_remove_1)
result_1 = get_crf_results(train_sents_1, val_sents_1, train_labels, val_labels)





#'stem', 'POS_tag'
features_remove_2 = ('Chunk_tag', 'is_BOS', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_2 = copy.deepcopy(train_sents)
val_sents_2 = copy.deepcopy(val_sents)
train_sents_2 = drop_dict_keys(train_sents_2, features_remove_2)
val_sents_2 = drop_dict_keys(val_sents_2, features_remove_2)
result_2 = get_crf_results(train_sents_2, val_sents_2, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 
features_remove_3 = ( 'is_BOS', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_3 = copy.deepcopy(train_sents)
val_sents_3 = copy.deepcopy(val_sents)
train_sents_3 = drop_dict_keys(train_sents_3, features_remove_3)
val_sents_3 = drop_dict_keys(val_sents_3, features_remove_3)
result_3 = get_crf_results(train_sents_3, val_sents_3, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_BOS',
features_remove_4 = ( 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_4 = copy.deepcopy(train_sents)
val_sents_4 = copy.deepcopy(val_sents)
train_sents_4 = drop_dict_keys(train_sents_4, features_remove_4)
val_sents_4 = drop_dict_keys(val_sents_4, features_remove_4)
result_4 = get_crf_results(train_sents_4, val_sents_4, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS',
features_remove_5 = ( 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_5 = copy.deepcopy(train_sents)
val_sents_5 = copy.deepcopy(val_sents)
train_sents_5 = drop_dict_keys(train_sents_5, features_remove_5)
val_sents_5 = drop_dict_keys(val_sents_5, features_remove_5)
result_5 = get_crf_results(train_sents_5, val_sents_5, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase',
features_remove_6 = ('word_shape', 'short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_6 = copy.deepcopy(train_sents)
val_sents_6 = copy.deepcopy(val_sents)
train_sents_6 = drop_dict_keys(train_sents_6, features_remove_6)
val_sents_6 = drop_dict_keys(val_sents_6, features_remove_6)
result_6 = get_crf_results(train_sents_6, val_sents_6, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 
features_remove_7 = ('short_word_shape','has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_7 = copy.deepcopy(train_sents)
val_sents_7 = copy.deepcopy(val_sents)
train_sents_7 = drop_dict_keys(train_sents_7, features_remove_7)
val_sents_7 = drop_dict_keys(val_sents_7, features_remove_7)
result_7 = get_crf_results(train_sents_7, val_sents_7, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape',
features_remove_8 = ('has_number','has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_8 = copy.deepcopy(train_sents)
val_sents_8 = copy.deepcopy(val_sents)
train_sents_8 = drop_dict_keys(train_sents_8, features_remove_8)
val_sents_8 = drop_dict_keys(val_sents_8, features_remove_8)
result_8 = get_crf_results(train_sents_8, val_sents_8, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number',
features_remove_9 = ('has_hyphen','is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_9 = copy.deepcopy(train_sents)
val_sents_9 = copy.deepcopy(val_sents)
train_sents_9 = drop_dict_keys(train_sents_9, features_remove_9)
val_sents_9 = drop_dict_keys(val_sents_9, features_remove_9)
result_9 = get_crf_results(train_sents_9, val_sents_9, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen',
features_remove_10 = ('is_upper_has_digit_has_dash',
            'prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_10 = copy.deepcopy(train_sents)
val_sents_10 = copy.deepcopy(val_sents)
train_sents_10 = drop_dict_keys(train_sents_10, features_remove_10)
val_sents_10 = drop_dict_keys(val_sents_10, features_remove_10)
result_10 = get_crf_results(train_sents_10, val_sents_10, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash',
features_remove_11 = ('prefix', 'suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_11 = copy.deepcopy(train_sents)
val_sents_11 = copy.deepcopy(val_sents)
train_sents_11 = drop_dict_keys(train_sents_11, features_remove_11)
val_sents_11 = drop_dict_keys(val_sents_11, features_remove_11)
result_11 = get_crf_results(train_sents_11, val_sents_11, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 
features_remove_12 = ('suffix','is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_12 = copy.deepcopy(train_sents)
val_sents_12 = copy.deepcopy(val_sents)
train_sents_12 = drop_dict_keys(train_sents_12, features_remove_12)
val_sents_12 = drop_dict_keys(val_sents_12, features_remove_12)
result_12 = get_crf_results(train_sents_12, val_sents_12, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix',
features_remove_13 = ('is_all_upper','is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_13 = copy.deepcopy(train_sents)
val_sents_13 = copy.deepcopy(val_sents)
train_sents_13 = drop_dict_keys(train_sents_13, features_remove_13)
val_sents_13 = drop_dict_keys(val_sents_13, features_remove_13)
result_13 = get_crf_results(train_sents_13, val_sents_13, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper',
features_remove_14 = ('is_stopword','word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_14 = copy.deepcopy(train_sents)
val_sents_14 = copy.deepcopy(val_sents)
train_sents_14 = drop_dict_keys(train_sents_14, features_remove_14)
val_sents_14 = drop_dict_keys(val_sents_14, features_remove_14)
result_14 = get_crf_results(train_sents_14, val_sents_14, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
features_remove_15 = ('word_previous', 'word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_15 = copy.deepcopy(train_sents)
val_sents_15 = copy.deepcopy(val_sents)
train_sents_15 = drop_dict_keys(train_sents_15, features_remove_15)
val_sents_15 = drop_dict_keys(val_sents_15, features_remove_15)
result_15 = get_crf_results(train_sents_15, val_sents_15, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 
features_remove_16 = ('word_previous_shape','word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_16 = copy.deepcopy(train_sents)
val_sents_16 = copy.deepcopy(val_sents)
train_sents_16 = drop_dict_keys(train_sents_16, features_remove_16)
val_sents_16 = drop_dict_keys(val_sents_16, features_remove_16)
result_16 = get_crf_results(train_sents_16, val_sents_16, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape',
features_remove_17 = ('word_previous_short_shape','word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_17 = copy.deepcopy(train_sents)
val_sents_17 = copy.deepcopy(val_sents)
train_sents_17 = drop_dict_keys(train_sents_17, features_remove_17)
val_sents_17 = drop_dict_keys(val_sents_17, features_remove_17)
result_17 = get_crf_results(train_sents_17, val_sents_17, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape', 'word_previous_short_shape',
features_remove_18 = ('word_after','word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_18 = copy.deepcopy(train_sents)
val_sents_18 = copy.deepcopy(val_sents)
train_sents_18 = drop_dict_keys(train_sents_18, features_remove_18)
val_sents_18 = drop_dict_keys(val_sents_18, features_remove_18)
result_18 = get_crf_results(train_sents_18, val_sents_18, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape', 'word_previous_short_shape', 'word_after',
features_remove_19 = ('word_after_shape','word_after_short_shape','is_in_gazetteer')
train_sents_19 = copy.deepcopy(train_sents)
val_sents_19 = copy.deepcopy(val_sents)
train_sents_19 = drop_dict_keys(train_sents_19, features_remove_19)
val_sents_19 = drop_dict_keys(val_sents_19, features_remove_19)
result_19 = get_crf_results(train_sents_19, val_sents_19, train_labels, val_labels)






#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape', 'word_previous_short_shape', 'word_after', 'word_after_shape',
features_remove_20 = ('word_after_short_shape','is_in_gazetteer')
train_sents_20 = copy.deepcopy(train_sents)
val_sents_20 = copy.deepcopy(val_sents)
train_sents_20 = drop_dict_keys(train_sents_20, features_remove_20)
val_sents_20 = drop_dict_keys(val_sents_20, features_remove_20)
result_20 = get_crf_results(train_sents_20, val_sents_20, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape', 'word_previous_short_shape', 'word_after', 'word_after_shape', 'word_after_short_shape',
features_remove_21 = ('is_in_gazetteer', 'is_in_gazetteer')
train_sents_21 = copy.deepcopy(train_sents)
val_sents_21 = copy.deepcopy(val_sents)
train_sents_21 = drop_dict_keys(train_sents_21, features_remove_21)
val_sents_21 = drop_dict_keys(val_sents_21, features_remove_21)
result_21 = get_crf_results(train_sents_21, val_sents_21, train_labels, val_labels)





#'stem', 'POS_tag', 'Chunk_tag', 'is_EOS', 'is_first_letter_uppercase', 'word_shape', 'short_word_shape','has_number','has_hyphen', 'is_upper_has_digit_has_dash', 'prefix', 'suffix','is_all_upper', 'is_stopword',
# 'word_previous', 'word_previous_shape', 'word_previous_short_shape', 'word_after', 'word_after_shape', 'word_after_short_shape', 'is_in_gazetteer'
features_remove_22 = ()
train_sents_22 = copy.deepcopy(train_sents)
val_sents_22 = copy.deepcopy(val_sents)
train_sents_22 = drop_dict_keys(train_sents_22, features_remove_22)
val_sents_22 = drop_dict_keys(val_sents_22, features_remove_22)
result_22 = get_crf_results(train_sents_22, val_sents_22, train_labels, val_labels)

[0.930163099892689, 0.9332775203457809, 0.9288045819990084]
[0.9580974997587028, 0.9588216969744169, 0.9578701400004289]
[0.9604951150014482, 0.9606713134223746, 0.9601071505128342]
[0.9600043134996975, 0.9603403294264242, 0.9597384972039107]
[0.9602500652615021, 0.9606323741287333, 0.9599931748925127]
[0.9675709307350736, 0.9669794789922511, 0.966903021502621]
[0.9693719917466549, 0.9694910634321093, 0.9692645097456983]
[0.9691546919098003, 0.969257427670262, 0.9690595203734476]
[0.9692768673217568, 0.9693158366107238, 0.9691387041066284]
[0.9692937027752458, 0.9693353062575445, 0.9691537641514449]
[0.9691918653839233, 0.969257427670262, 0.9690714504498199]
[0.971676373929788, 0.9718468906974027, 0.9716730303301021]
[0.9723147521160607, 0.9726062069234064, 0.972400137126605]
[0.9723838702581897, 0.9726646158638682, 0.9724634936188514]
[0.972330372673442, 0.972625676570227, 0.9724165451660205]
[0.9754232897023084, 0.9757602897083447, 0.975528555960034]
[0.9765209019662725, 0.9768116506

In [39]:
print(train_sents_1[0][0])
print(train_sents_2[0][0])
print(train_sents_3[0][0])
print(train_sents_4[0][0])
print(train_sents_5[0][0])
print(train_sents_6[0][0])
print(train_sents_7[0][0])
print(train_sents_8[0][0])
print(train_sents_9[0][0])
print(train_sents_10[0][0])
print(train_sents_11[0][0])
print(train_sents_12[0][0])
print(train_sents_13[0][0])
print(train_sents_14[0][0])
print(train_sents_15[0][0])
print(train_sents_16[0][0])
print(train_sents_17[0][0])
print(train_sents_18[0][0])
print(train_sents_19[0][0])
print(train_sents_20[0][0])
print(train_sents_21[0][0])
print(train_sents_22[0][0])

{'stem': 'eu'}
{'stem': 'eu', 'POS_tag': 'NNP'}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP'}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False, 'is_first_letter_uppercase': True}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False, 'is_first_letter_uppercase': True, 'word_shape': 'XX'}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False, 'is_first_letter_uppercase': True, 'word_shape': 'XX', 'short_word_shape': 'X'}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False, 'is_first_letter_uppercase': True, 'word_shape': 'XX', 'short_word_shape': 'X', 'has_number': False}
{'stem': 'eu', 'POS_tag': 'NNP', 'Chunk_tag': 'B-NP', 'is_BOS': True, 'is_EOS': False, 'is_first_letter_uppercase': True, 

In [40]:
# start from the stem of the token and add features one by one and train a new model with each feature that you add


features_for_add_1by1 = ['Stem', 
            '+POS_tag', 
            '+Chunk_tag', 
            '+is_BOS', 
            '+is_EOS', 
            '+is_first_letter_uppercase', 
            '+word_shape', 
            '+short_word_shape',
            '+has_number',
            '+has_hyphen',
            '+is_upper_has_digit_has_dash',
            '+prefix',
            '+suffix',
            '+is_all_upper',
            '+is_stopword',
            '+word_previous',
            '+word_previous_shape',
            '+word_previous_short_shape',
            '+word_after',
            '+word_after_shape',
            '+word_after_short_shape',
            '+is_in_gazetteer']



results_features_1by1_df = pd.DataFrame(columns = ["Precision", "Recall", "F1 Score"])
series_1 = pd.Series(result_1, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_1, ignore_index=True)

series_2 = pd.Series(result_2, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_2, ignore_index=True)

series_3 = pd.Series(result_3, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_3, ignore_index=True)

series_4 = pd.Series(result_4, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_4, ignore_index=True)

series_5 = pd.Series(result_5, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_5, ignore_index=True)

series_6 = pd.Series(result_6, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_6, ignore_index=True)

series_7 = pd.Series(result_7, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_7, ignore_index=True)

series_8 = pd.Series(result_8, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_8, ignore_index=True)

series_9 = pd.Series(result_9, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_9, ignore_index=True)

series_10 = pd.Series(result_10, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_10, ignore_index=True)

series_11 = pd.Series(result_11, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_11, ignore_index=True)

series_12 = pd.Series(result_12, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_12, ignore_index=True)

series_13 = pd.Series(result_13, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_13, ignore_index=True)

series_14 = pd.Series(result_14, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_14, ignore_index=True)

series_15 = pd.Series(result_15, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_15, ignore_index=True)

series_16 = pd.Series(result_16, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_16, ignore_index=True)

series_17 = pd.Series(result_17, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_17, ignore_index=True)

series_18 = pd.Series(result_18, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_18, ignore_index=True)

series_19 = pd.Series(result_19, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_19, ignore_index=True)

series_20 = pd.Series(result_20, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_20, ignore_index=True)

series_21 = pd.Series(result_21, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_21, ignore_index=True)

series_22 = pd.Series(result_22, index = results_features_1by1_df.columns)
results_features_1by1_df = results_features_1by1_df.append(series_22, ignore_index=True)

###Results Table of the Effect of Each Feature to the CRF Model

In [41]:
# display the result table
results_features_1by1_df.insert(0, "Features", features_for_add_1by1)
results_features_1by1_df

,Features,Precision,Recall,F1 Score
0,Stem,0.930163,0.933278,0.928805
1,+POS_tag,0.958097,0.958822,0.957870
2,+Chunk_tag,0.960495,0.960671,0.960107
3,+is_BOS,0.960004,0.960340,0.959738
4,+is_EOS,0.960250,0.960632,0.959993
5,+is_first_letter_uppercase,0.967571,0.966979,0.966903
6,+word_shape,0.969372,0.969491,0.969265
7,+short_word_shape,0.969155,0.969257,0.969060
8,+has_number,0.969277,0.969316,0.969139
9,+has_hyphen,0.969294,0.969335,0.969154


In [42]:
# Average Precision - Recall - F1 Score for Features
avg_result = 0
feature = ""
features_avg = []
feature_names = []
for i in range(results_features_1by1_df.shape[0]):
  for j in range(results_features_1by1_df.shape[1]):
    if j == 0:
      feature = results_features_1by1_df.loc[i][0]
      feature_names.append(feature)
    if j > 0:
      avg_result += results_features_1by1_df.loc[i][j]
  avg_result = avg_result/3
  print("Feature ["+ feature + "] overall result: ", avg_result)
  
  features_avg.append(avg_result)
  avg_result = 0

Feature [Stem] overall result:  0.9307484007458261
Feature [+POS_tag] overall result:  0.9582631122445161
Feature [+Chunk_tag] overall result:  0.960424526312219
Feature [+is_BOS] overall result:  0.9600277133766775
Feature [+is_EOS] overall result:  0.9602918714275828
Feature [+is_first_letter_uppercase] overall result:  0.9671511437433152
Feature [+word_shape] overall result:  0.9693758549748209
Feature [+short_word_shape] overall result:  0.9691572133178367
Feature [+has_number] overall result:  0.9692438026797031
Feature [+has_hyphen] overall result:  0.969260924394745
Feature [+is_upper_has_digit_has_dash] overall result:  0.9691735811680017
Feature [+prefix] overall result:  0.9717320983190976
Feature [+suffix] overall result:  0.9724403653886907
Feature [+is_all_upper] overall result:  0.9725039932469698
Feature [+is_stopword] overall result:  0.9724575314698966
Feature [+word_previous] overall result:  0.9755707117902291
Feature [+word_previous_shape] overall result:  0.9766463

In [43]:
# Contribution of each feature after added to the model
feature_diffs = []
features_avg.insert(0, 0)
for i in range(1,len(features_avg)):
    x = features_avg[i] - features_avg[i-1]
    feature_diffs.append(round(x,4))
features_avg.pop(0)
features_effect_to_score = list(zip(feature_names, feature_diffs))
print("Contribution of features when added to the model:")
print(features_effect_to_score)

Contribution of features when added to the model:
[('Stem', 0.9307), ('+POS_tag', 0.0275), ('+Chunk_tag', 0.0022), ('+is_BOS', -0.0004), ('+is_EOS', 0.0003), ('+is_first_letter_uppercase', 0.0069), ('+word_shape', 0.0022), ('+short_word_shape', -0.0002), ('+has_number', 0.0001), ('+has_hyphen', 0.0), ('+is_upper_has_digit_has_dash', -0.0001), ('+prefix', 0.0026), ('+suffix', 0.0007), ('+is_all_upper', 0.0001), ('+is_stopword', -0.0), ('+word_previous', 0.0031), ('+word_previous_shape', 0.0011), ('+word_previous_short_shape', -0.0), ('+word_after', 0.0012), ('+word_after_shape', 0.001), ('+word_after_short_shape', 0.0002), ('+is_in_gazetteer', -0.0001)]


In [44]:
# Sort contributions of each features score
contribution_of_scores = copy.deepcopy(features_effect_to_score)
contribution_of_scores = sorted(contribution_of_scores, key=lambda tup: tup[1], reverse=True)

print("Contribution of features [Sorted in descending order]")
contribution_of_scores

Contribution of features [Sorted in descending order]


[('Stem', 0.9307),
 ('+POS_tag', 0.0275),
 ('+is_first_letter_uppercase', 0.0069),
 ('+word_previous', 0.0031),
 ('+prefix', 0.0026),
 ('+Chunk_tag', 0.0022),
 ('+word_shape', 0.0022),
 ('+word_after', 0.0012),
 ('+word_previous_shape', 0.0011),
 ('+word_after_shape', 0.001),
 ('+suffix', 0.0007),
 ('+is_EOS', 0.0003),
 ('+word_after_short_shape', 0.0002),
 ('+has_number', 0.0001),
 ('+is_all_upper', 0.0001),
 ('+has_hyphen', 0.0),
 ('+is_stopword', -0.0),
 ('+word_previous_short_shape', -0.0),
 ('+is_upper_has_digit_has_dash', -0.0001),
 ('+is_in_gazetteer', -0.0001),
 ('+short_word_shape', -0.0002),
 ('+is_BOS', -0.0004)]

###CLassification Report of the Best Model

In [45]:
# Displaying the classification report for the best model
features_remove_for_best_model = ('has_hyphen', 'word_previous_short_shape', 'is_in_gazetteer','is_upper_has_digit_has_dash', 'is_stopword', 'short_word_shape', 'is_BOS')
train_sents_for_best_model = copy.deepcopy(train_sents)
val_sents_for_best_model = copy.deepcopy(val_sents)
test_sents_for_best_model = copy.deepcopy(test_sents)

train_sents_for_best_model = drop_dict_keys(train_sents_for_best_model, features_remove_for_best_model)
val_sents_for_best_model = drop_dict_keys(val_sents_for_best_model, features_remove_for_best_model)
test_sents_for_best_model = drop_dict_keys(test_sents_for_best_model, features_remove_for_best_model)

overall_train_sents_model = train_sents_for_best_model + val_sents_for_best_model
overall_train_labels_model = train_labels + val_labels



crf_best_model_estimator = sklearn_crfsuite.CRF(c1=0.05, c2=0.25, keep_tempfiles=None)
crf_best_model_estimator.fit(overall_train_sents_model, overall_train_labels_model)
crf_best_model_estimator_preds = crf_best_model_estimator.predict(test_sents_for_best_model)

precisions_crf_best_model = sklearn_crfsuite.metrics.flat_precision_score(test_labels, crf_best_model_estimator_preds, average = 'weighted')
recalls_crf_best_model = sklearn_crfsuite.metrics.flat_recall_score(test_labels, crf_best_model_estimator_preds, average = 'weighted')
f1_scores_crf_best_model = sklearn_crfsuite.metrics.flat_f1_score(test_labels, crf_best_model_estimator_preds, average = 'weighted')



print("\nResults of the Best Model: "+ "\n"+
      "--------------------------------------------")
print("Precision:", precisions_crf_best_model)
print("Recall: ", recalls_crf_best_model)
print("F1 Score: ", f1_scores_crf_best_model)


Results of the Best Model: 
--------------------------------------------
Precision: 0.9633327236178337
Recall:  0.9619683428448369
F1 Score:  0.9624381127689882


## Recurrent Neural Network (RNN)

###Libraries

In [38]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Dropout, LSTM, TimeDistributed, Bidirectional, SimpleRNN
from keras.callbacks import ModelCheckpoint

from gensim.models import Word2Vec
import gensim.downloader as api
import copy

In [39]:
!pip install seqeval
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Prepare the Datasets by DeepCopy

In [40]:
# Prepare DeepCopies of train_sents, train_labels, val_sent, val_labels (just in case)
train_sents_rnn = copy.deepcopy(train_sents)
train_labels_rnn = copy.deepcopy(train_labels)

val_sents_rnn = copy.deepcopy(val_sents)
val_labels_rnn = copy.deepcopy(val_labels)

test_sents_rnn = copy.deepcopy(test_sents)
test_labels_rnn = copy.deepcopy(test_labels)

###Find MAX_LENGTH's for Padding

In [41]:
# Find Word and Label with Max Length
def FIND_MAX_LENGTH_WORD(data_sents):
  MAX_SENT_LENGTH = 0
  for sentence in data_sents:
    if len(sentence) > MAX_SENT_LENGTH:
      MAX_SENT_LENGTH = len(sentence)

  return MAX_SENT_LENGTH



def FIND_MAX_LENGTH_LABEL(data_labels):
  MAX_LENGTH_LABEL = 0
  for sentence in data_labels:
    if len(sentence) > MAX_LENGTH_LABEL:
      MAX_LENGTH_LABEL = len(sentence)

  return MAX_LENGTH_LABEL



MAX_LENGTH_train_sents = FIND_MAX_LENGTH_WORD(train_sents_rnn)
MAX_LENGTH_train_labels = FIND_MAX_LENGTH_LABEL(train_labels_rnn)

MAX_LENGTH_val_sents = FIND_MAX_LENGTH_WORD(val_sents_rnn)
MAX_LENGTH_val_labels = FIND_MAX_LENGTH_LABEL(val_labels_rnn)

MAX_LENGTH_test_sents = FIND_MAX_LENGTH_WORD(test_sents_rnn)
MAX_LENGTH_test_labels = FIND_MAX_LENGTH_LABEL(test_labels_rnn)

MAX_LENGTH_all_sents = max(MAX_LENGTH_train_sents, MAX_LENGTH_val_sents, MAX_LENGTH_test_sents)
MAX_LENGTH_all_labels = max(MAX_LENGTH_train_labels, MAX_LENGTH_val_labels, MAX_LENGTH_test_labels)

MAX_LENGTH_overall = max(MAX_LENGTH_all_sents, MAX_LENGTH_all_labels)

print("Length of the Longest Sentences:", MAX_LENGTH_all_sents)
print("Length of the Longest Label:", MAX_LENGTH_all_labels)
print("Max Length:", MAX_LENGTH_overall)

Length of the Longest Sentences: 124
Length of the Longest Label: 124
Max Length: 124


###Find Unique Labels

In [42]:
# find unique labels and create dictionary to map each label to a unique integer value
UniqueLabels = {}

for sent in train_labels_rnn:
  for i in sent: 
    UniqueLabels[i] = ""

for sent in val_labels_rnn:
  for i in sent: 
    UniqueLabels[i] = ""

for sent in test_labels_rnn:
  for i in sent: 
    UniqueLabels[i] = ""

print("Unique labels in the dictionary: ", UniqueLabels.keys())

Unique labels in the dictionary:  dict_keys(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC'])


###Map Labels to Integers

In [43]:

# Function for mapping values; 5 --> 4, 4 --> 3, 3 --> 2, 2 --> 1, 1 --> 0
labels_map = {'O': 1, 'B-ORG': 2, 'B-MISC': 3, 'B-PER': 4, 'I-PER': 5, 'B-LOC': 6, 'I-ORG': 7, 'I-MISC': 8, 'I-LOC': 9}

labels_to_list_map = {
  1: [1,0,0,0,0,0,0,0,0], 2: [0,1,0,0,0,0,0,0,0], 3: [0,0,1,0,0,0,0,0,0], 4: [0,0,0,1,0,0,0,0,0],
  5: [0,0,0,0,1,0,0,0,0], 6: [0,0,0,0,0,1,0,0,0], 7: [0,0,0,0,0,0,1,0,0], 8: [0,0,0,0,0,0,0,1,0],
  9: [0,0,0,0,0,0,0,0,1], 0: [0,0,0,0,0,0,0,0,0]
}

# Turn labels into numeric format
# Reference: https://stackoverflow.com/questions/69411748/map-elements-of-a-list-of-lists-to-a-dictionary-value
def DEEP_MAP(func, dataset_):
    if not isinstance(dataset_, list):
        return func(dataset_)
    return [DEEP_MAP(func, x) for x in dataset_]

train_labels_rnn = DEEP_MAP(lambda x: labels_map[x], train_labels_rnn)
val_labels_rnn = DEEP_MAP(lambda x: labels_map[x], val_labels_rnn)
test_labels_rnn = DEEP_MAP(lambda x: labels_map[x], test_labels_rnn)

train_labels_rnn = DEEP_MAP(lambda x: labels_to_list_map[x], train_labels_rnn)
val_labels_rnn = DEEP_MAP(lambda x: labels_to_list_map[x], val_labels_rnn)
#test_labels_rnn = DEEP_MAP(lambda x: labels_to_list_map[x], test_labels_rnn)


print("Train Labels After Mapping:\n", train_labels_rnn[:3])
print("Validation Labels After Mapping:\n", val_labels_rnn[:3])
print("Test Labels After Mapping:\n", test_labels_rnn[:3])

Train Labels After Mapping:
 [[[0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]]]
Validation Labels After Mapping:
 [[[1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 

###Turn Dataset into List of List Format

In [44]:
# Turn datasets into list of list format instead of list of list of dict
# Just apply this on sents, since labels are already in list of lists format 
def LIST_OF_LIST_SENTS(data_sents):
  temp_paragraph = []
  temp_sent = []
  for sentence in data_sents:
    for word in sentence:
      stem = word.get('stem')
      temp_sent.append(stem)
    temp_paragraph.append(temp_sent)
    temp_sent = []

  return temp_paragraph

# train_sents
train_sents_rnn_lol = LIST_OF_LIST_SENTS(train_sents_rnn)

# val_sents
val_sents_rnn_lol = LIST_OF_LIST_SENTS(val_sents_rnn)

# test_sents
test_sents_rnn_lol = LIST_OF_LIST_SENTS(test_sents_rnn)

###Tokenize the Sentences

In [45]:
# Tokenizing sentences and labels
tokenizer_keras_rnn = Tokenizer(filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'')

# train_sents
tokenizer_keras_rnn.fit_on_texts(train_sents_rnn_lol)
train_sents_rnn_tok = tokenizer_keras_rnn.texts_to_sequences(train_sents_rnn_lol)

# val_sents
tokenizer_keras_rnn.fit_on_texts(val_sents_rnn_lol)
val_sents_rnn_tok = tokenizer_keras_rnn.texts_to_sequences(val_sents_rnn_lol)

# test_sents
tokenizer_keras_rnn.fit_on_texts(test_sents_rnn_lol)
test_sents_rnn_tok = tokenizer_keras_rnn.texts_to_sequences(test_sents_rnn_lol)

print("Tokenized Train Sents:", train_sents_rnn_tok[:3])
print("Tokenized Validation Sents:", val_sents_rnn_tok[:3])
print("Tokenized Test Sents:", test_sents_rnn_tok[:3])

Tokenized Train Sents: [[1084, 2168, 230, 185, 6, 3570, 252, 5029, 2], [852, 1884], [830, 173]]
Tokenized Validation Sents: [[349, 17, 1716, 142, 65, 18, 419, 41, 5, 327, 2], [116, 293], [304, 791, 3690, 2784, 3676, 303, 168, 15, 1564, 12, 78, 31, 1716, 53, 1884, 28, 36, 227, 8, 1736, 152, 5, 49, 95, 6, 142, 65, 18, 1, 410, 4, 1, 665, 244, 2]]
Tokenized Test Sents: [[86, 16, 243, 407, 3862, 115, 2, 204, 5, 1465, 612, 3], [20034, 20035], [8102, 2, 134, 831, 4701, 206]]


###Padding the Labels and Sentences

In [46]:
# preprare your dataset for RNN classifier (you need to add padding to labels as well)
# Padding sentences and labels

# train_sents
padded_train_sents_rnn = pad_sequences(train_sents_rnn_tok, padding='post', maxlen=MAX_LENGTH_overall)

# train_labels
padded_train_labels_rnn = pad_sequences(train_labels_rnn, padding='post', maxlen=MAX_LENGTH_overall)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

# val_sents
padded_val_sents_rnn = pad_sequences(val_sents_rnn_tok, padding='post', maxlen=MAX_LENGTH_overall)

# val_labels
padded_val_labels_rnn = pad_sequences(val_labels_rnn, padding='post', maxlen=MAX_LENGTH_overall)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

# test_sents
padded_test_sents_rnn = pad_sequences(test_sents_rnn_tok, padding='post', maxlen=MAX_LENGTH_overall)

# test_labels
padded_test_labels_rnn = pad_sequences(test_labels_rnn, padding='post', maxlen=MAX_LENGTH_overall)


print("Padded Train Sents:\n", padded_train_sents_rnn[:1], "\n")
print("Padded Train Labels:\n", padded_train_labels_rnn[:1], "\n")
print("-------------------------------------------------------------------------")
print("Padded Validation Sents:\n", padded_val_sents_rnn[:1], "\n")
print("Padded Validation Labels:\n", padded_val_labels_rnn[:1], "\n")
print("-------------------------------------------------------------------------")
print("Padded Test Sents:\n", padded_test_sents_rnn[:1], "\n")
print("Padded Test Labels:\n", padded_test_labels_rnn[:1])

Padded Train Sents:
 [[1084 2168  230  185    6 3570  252 5029    2    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]] 

Padded Train Labels:
 [[[0 1 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]] 

-------------------------------------------------------------------------
Padded Validation Sents:
 [[ 349   17 1716  142   65   18  419   41    5  327    2    0    0    0
     0    0  

In [47]:
padded_train_sents_rnn.shape

(14041, 124)

###Create the Embedding Layers

In [48]:
# RANDOMLY INITIALIZED
random_emb_layer_rnn = Embedding(input_dim=len(tokenizer_keras_rnn.word_index)+1, output_dim=64, input_length=MAX_LENGTH_overall)

In [49]:
# FROM SCRATCH
my_rnn_embedding = Word2Vec(sentences = train_sents_rnn_lol, min_count=1)
my_rnn_emb_layer = Embedding(input_dim=len(my_rnn_embedding.wv.vocab), output_dim=100, input_length=MAX_LENGTH_overall, weights=[my_rnn_embedding.wv.vectors])

In [50]:
# Create your own word embeddings from scratch and load a pretrained word embeddings
# You can check https://radimrehurek.com/gensim/models/word2vec.html for training a word embeddings from scratch
# You can check https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html and https://github.com/RaRe-Technologies/gensim-data for loading pretrained word embeddings.

# PRETRAINED
pretrained_rnn_embedding = Word2Vec(sentences=api.load('text8'), min_count=1)
pretrained_rnn_emb_layer = Embedding(input_dim=len(pretrained_rnn_embedding.wv.vocab), output_dim=100, input_length=MAX_LENGTH_overall, weights=[pretrained_rnn_embedding.wv.vectors])

[==================================================] 100.0% 31.6/31.6MB downloaded


###Create the RNN Models

In [51]:
# Turn labels into numeric format
# Reference: https://stackoverflow.com/questions/69411748/map-elements-of-a-list-of-lists-to-a-dictionary-value
def DEEP_MAP(func, dataset_):
    if not isinstance(dataset_, list):
        return func(dataset_)
    return [DEEP_MAP(func, x) for x in dataset_]

In [52]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
  d = []
  for label_list in encoded_seq[0]:
    e = []
    for vector in label_list:
      e.append(np.argmax(vector) + 1)
    d.append(e)
  return d
# define a function to remove paddings and align labels and tokens

In [53]:
def reverse_map(sequence):
  # Function for mapping values; 5 --> 4, 4 --> 3, 3 --> 2, 2 --> 1, 1 --> 0
  reverse_labels_map = {1: 'O', 2: 'B-ORG', 3: 'B-MISC', 4: 'B-PER', 5: 'I-PER', 6: 'B-LOC', 7: 'I-ORG', 8: 'I-MISC', 9:'I-LOC'}

  sequence =  DEEP_MAP(lambda x: reverse_labels_map[x], sequence)
  return sequence

####SimpleRNN Model

In [61]:
def RNN_Model_Generator(op_name, embedding_layer_opt, rnn_layer_size, hidden_layer_size, 
                            padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn):

    print(op_name)
    print("=================================")
    print("RNN Hidden Layer Size: ", rnn_layer_size)
    print("Hidden Layer Size: ", hidden_layer_size)
    print("--------------------------------------------------------------------- \n")


    # Define the Model
    # --------------------------------------------------------------------------
    model = Sequential()

    # Add Embedding Layer
    model.add(embedding_layer_opt)

    # Add RNN
    model.add(SimpleRNN(units=rnn_layer_size, return_sequences=True))

    model.add(Dropout(rate=0.02))

    # Add Dense Layer
    model.add(Dense(hidden_layer_size, activation='relu'))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(9, activation="softmax")))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    print("\n--------------------------------------------------------------------- \n")

    model.fit(padded_train_sents_rnn, padded_train_labels_rnn, epochs=10, verbose=1)
    pred_labels = model.predict(padded_val_sents_rnn)

    pred_labels_new = one_hot_decode(pred_labels)
    pred_labels_newest = reverse_map(pred_labels_new)

    padded_val_labels_rnn_new = one_hot_decode(padded_val_labels_rnn)
    padded_val_labels_rnn_newest = reverse_map(padded_val_labels_rnn_new)
    print("F1 Score: {:.4%}".format(f1_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Precision Score: {:.4%}".format(precision_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Recall Score: {:.4%}".format(recall_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("\n\n\n")

####LSTM Model

In [62]:
def LSTM_Model_Generator(op_name, embedding_layer_opt, rnn_layer_size, hidden_layer_size, 
                            padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn):
  
    print(op_name)
    print("=================================")
    print("RNN Hidden Layer Size: ", rnn_layer_size)
    print("Hidden Layer Size: ", hidden_layer_size)
    print("--------------------------------------------------------------------- \n")

    model = Sequential()

    # Add Embedding layer
    model.add(embedding_layer_opt)

    # Add LSTM
    model.add
    model.add(LSTM(units=rnn_layer_size, return_sequences=True))

    model.add(Dropout(rate=0.02))

    model.add(Dense(hidden_layer_size, activation='relu'))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(9, activation="softmax")))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    model.fit(padded_train_sents_rnn, padded_train_labels_rnn, epochs=10, verbose=1)
    pred_labels = model.predict(padded_val_sents_rnn)

    pred_labels_new = one_hot_decode(pred_labels)
    pred_labels_newest = reverse_map(pred_labels_new)

    padded_val_labels_rnn_new = one_hot_decode(padded_val_labels_rnn)
    padded_val_labels_rnn_newest = reverse_map(padded_val_labels_rnn_new)
    print("F1 Score: {:.4%}".format(f1_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Precision Score: {:.4%}".format(precision_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Recall Score: {:.4%}".format(recall_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("\n\n\n")

####Bidirectional LSTM Model

In [63]:
# Create your models and train them
# RNN Model Reference: https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54

def Bi_LSTM_Model_Generator(op_name, embedding_layer_opt, rnn_layer_size, hidden_layer_size, 
                            padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn):
  
    print(op_name)
    print("=================================")
    print("RNN Hidden Layer Size: ", rnn_layer_size)
    print("Hidden Layer Size: ", hidden_layer_size)
    print("--------------------------------------------------------------------- \n")

    model = Sequential()
    # Add Embedding layer
    model.add(embedding_layer_opt)

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=rnn_layer_size, return_sequences=True)))

    model.add(Dropout(rate=0.02))

    model.add(Dense(hidden_layer_size, activation='relu'))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(9, activation="softmax")))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
  
    model.fit(padded_train_sents_rnn, padded_train_labels_rnn, epochs=10, verbose=1)
    pred_labels = model.predict(padded_val_sents_rnn)

    pred_labels_new = one_hot_decode(pred_labels)
    pred_labels_newest = reverse_map(pred_labels_new)

    padded_val_labels_rnn_new = one_hot_decode(padded_val_labels_rnn)
    padded_val_labels_rnn_newest = reverse_map(padded_val_labels_rnn_new)
    print("F1 Score: {:.4%}".format(f1_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Precision Score: {:.4%}".format(precision_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("Recall Score: {:.4%}".format(recall_score(padded_val_labels_rnn_newest, pred_labels_newest)))
    print("\n\n\n")

###Generate the Models

In [57]:
import numpy as np

In [59]:
# General
operation_names = ['Random Embedded Model', 'Embedded From Scratch Model', 'Pretrained Embedding Model']
embedding_layer_options = [random_emb_layer_rnn, my_rnn_emb_layer, pretrained_rnn_emb_layer]
RNN_layer_size_options = [32, 64]
hidden_layer_options =[16, 24]


In [64]:
# Random Embedded Model
for a in range(len(RNN_layer_size_options)):
  for b in range(len(hidden_layer_options)):
    RNN_Model_Generator('Random Embedded Model', random_emb_layer_rnn, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    LSTM_Model_Generator('Random Embedded Model', random_emb_layer_rnn, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    Bi_LSTM_Model_Generator('Random Embedded Model', random_emb_layer_rnn, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)

Random Embedded Model
RNN Hidden Layer Size:  32
Hidden Layer Size:  16
--------------------------------------------------------------------- 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 124, 64)           1409024   
                                                                 
 simple_rnn (SimpleRNN)      (None, 124, 32)           3104      
                                                                 
 dropout_1 (Dropout)         (None, 124, 32)           0         
                                                                 
 dense_2 (Dense)             (None, 124, 16)           528       
                                                                 
 time_distributed_1 (TimeDis  (None, 124, 9)           153       
 tributed)                                                       
                                          

In [ ]:
# Embedded From Scratch Model
for a in range(len(RNN_layer_size_options)):
  for b in range(len(hidden_layer_options)):
    RNN_Model_Generator('Embedded From Scratch Model', my_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    LSTM_Model_Generator('Embedded From Scratch Model', my_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    Bi_LSTM_Model_Generator('Embedded From Scratch Model', my_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)

In [ ]:
# Pretrained Embedding Model
for a in range(len(RNN_layer_size_options)):
  for b in range(len(hidden_layer_options)):
    RNN_Model_Generator('Pretrained Embedding Model', pretrained_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    LSTM_Model_Generator('Pretrained Embedding Model', pretrained_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)
    Bi_LSTM_Model_Generator('Pretrained Embedding Model', pretrained_rnn_emb_layer, RNN_layer_size_options[a], hidden_layer_options[b], padded_train_sents_rnn, padded_train_labels_rnn, 
                            padded_val_sents_rnn, padded_val_labels_rnn, 
                            padded_test_sents_rnn, padded_test_labels_rnn)

In [ ]:
padded_overall_sents = np.concatenate((padded_train_sents_rnn, padded_val_sents_rnn), axis=0)
padded_overall_labels = np.concatenate((padded_train_labels_rnn, padded_val_labels_rnn), axis=0)

In [ ]:
# Evaluate your models with functions of seqeval library
# Since every model generates the same results, it is useless to make evaluation on test data using (train + validation) data.

## My Report

Named Entity Recognition task carried out best with CRF model with very high F1, Precision, and Recall Score (96-97%). Feature extraction step and eliminating the features which has no contribution or the features which decreases the score of the model, improves the overall score of the model in the test data. 

In the RNN part of the project, due to an "overfitting" situation experienced with model; the F1, Precision, and Recall Scores are very very low. I think this situation is caused by the zero's in the data due to padding. Due to the majority of zero's in the data, RNN models assummed that, marking every word with "zero" which corresponds to the "Other" label provide the best predictions. This conclusion is derived from the epochs. In the epochs, accuracy of the model's are displayed around 93 - 96%. But, when the time for testing the model on the validation set comes, the model always predict the label as zero.

An analogy about this situation can be formed as the following:
If we provide 90 cat images and 10 dog images to a model and expect model to predict the images we give as the test set, the model will predict every image as "cat" in order to work with 90% accuracy.

As the result of these experiments it can ben concluded that, CRF models work best for NER task and RNN - LSTM models are not useful for this task.



##NOTE ABOUT SUBMISSION: 
I submitted this homework 2 days late because in the first day I was trying to fix some errors and in the 2nd day even if I start running the code in the afternoon, at the middle of the execution somehow my dorm internet get disconnected. Because of that I re-run the code but the code couldn't finish the execution unyil the midnight. Today, I couldn't run the models with "Embedded From Scratch Model" and "Pretrained Embedding Model" because runtime is disconnected and my GPU limit is finished when I was having my CS412 Machine Learning Final. 